In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
overwrite = True

In [4]:
if overwrite:
    param_df = pd.read_csv('parameters-template.csv')
else:
    param_df = pd.read_csv('parameters.csv')

In [5]:
param_df

,Sim,CoefL,CoefT,Gain,GETGain,PeakingTime,N,P0,E0,P1,E1,Status
0,800p_Ref,0.000114,0.00037,10000,1000,720,100,p,800,0,0,0


In [ ]:
P0, E0 = 'p', 800
P1, E1 = '0', 0

parameter_guesses = {
    'CoefL' : 0.000114,
    'CoefT' : 0.00037,
    'Gain' : 10000,
    'GETGain' : 1000,
    'PeakingTime' : 720,
    'N' : 100}
